# Capa Silver — Limpieza y estandarización de `fighter_details`
**Proyecto:** UFC — ETL de Raw → Bronze → Silver  
**Notebook:** `load_bronze_to_silver.ipynb`  
**Objetivo:** convertir el dataset de Bronze en un dataset **limpio, tipado y coherente** (Silver) listo para análisis.

### Alcance
- Limpieza de `Weight` y `Height`
- Creación de `Weight_Class`
- Conversión de `DOB` → **Age** (edad al 21/03/2021)
- Tipado de métricas (float64)
- Eliminación de filas sin datos clave y outliers
- Trazabilidad con `_SilverTimestamp`
- Guardado de **Parquet** en `data/silver/`

> **Fecha de corte de los datos:** 21 de marzo de 2021



## Imports

In [198]:
# Cargamos librerías necesarias para limpieza y manejo de rutas.
import os
from datetime import datetime
import pandas as pd
import numpy as np
import re

## Instalacion fastparquet

In [199]:
%pip install fastparquet


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Definición de rutas

Se configuran las rutas y nombres de archivos para leer los datos desde la capa **Bronze**
y guardar la versión limpia en la capa **Silver**.

In [200]:
# Definimos la ruta relativa a la carpeta donde están los archivos Bronze y Silver
# (usamos os.curdir y subimos un nivel con '..' porque el notebook está en /notebooks)
bronze_path = os.path.join(os.curdir, "..", "data", "bronze")
silver_path = os.path.join(os.curdir, "..", "data", "silver")

# Nombres de los archivos parquet para cada capa
bronze_file_name = "fighter_details_bronze.parquet"
silver_file_name = "fighter_details_silver.parquet"

# Construimos las rutas completas de los archivos parquet
bronze_file_path = os.path.join(bronze_path, bronze_file_name)
silver_file_path = os.path.join(silver_path, silver_file_name)

os.makedirs(silver_path, exist_ok=True)


## Carga del dataset Bronze

Leemos el archivo `fighter_details_bronze.parquet` para inspeccionar la estructura de los datos
previos a la limpieza y transformación hacia la capa **Silver**.

In [201]:
# Leemos el archivo parquet que contiene los datos en estado Bronze
# (dataset ya transformado desde la capa Raw)
bronze_df = pd.read_parquet(bronze_file_path)

# Mostramos la forma del DataFrame: número de filas y columnas
print("Dimensiones del dataset Bronze:", bronze_df.shape)

# Visualizamos las primeras filas para revisar estructura y columnas
bronze_df.head()

Dimensiones del dataset Bronze: (3596, 15)


,fighter_name,Height,Weight,Reach,Stance,DOB,SLpM,Str_Acc,SApM,Str_Def,TD_Avg,TD_Acc,TD_Def,Sub_Avg,_BronzeTimestamp
0,Tom Aaron,None,155 lbs.,None,None,"Jul 13, 1978",0.00,0%,0.00,0%,0.00,0%,0%,0.0,2025-11-10 13:49:51.311624
1,Papy Abedi,"5' 11""",185 lbs.,None,Southpaw,"Jun 30, 1978",2.80,55%,3.15,48%,3.47,57%,50%,1.3,2025-11-10 13:49:51.311624
2,Shamil Abdurakhimov,"6' 3""",235 lbs.,"76""",Orthodox,"Sep 02, 1981",2.45,44%,2.45,58%,1.23,24%,47%,0.2,2025-11-10 13:49:51.311624
3,Danny Abbadi,"5' 11""",155 lbs.,None,Orthodox,"Jul 03, 1983",3.29,38%,4.41,57%,0.00,0%,77%,0.0,2025-11-10 13:49:51.311624
4,Hiroyuki Abe,"5' 6""",145 lbs.,None,Orthodox,None,1.71,36%,3.11,63%,0.00,0%,33%,0.0,2025-11-10 13:49:51.311624


## Tratar Nulos

In [202]:
# Compruebo los nulos en cada columna
bronze_df.info()
bronze_df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3596 entries, 0 to 3595
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fighter_name      3596 non-null   object        
 1   Height            3333 non-null   object        
 2   Weight            3522 non-null   object        
 3   Reach             1684 non-null   object        
 4   Stance            2792 non-null   object        
 5   DOB               2857 non-null   object        
 6   SLpM              3596 non-null   float64       
 7   Str_Acc           3596 non-null   object        
 8   SApM              3596 non-null   float64       
 9   Str_Def           3596 non-null   object        
 10  TD_Avg            3596 non-null   float64       
 11  TD_Acc            3596 non-null   object        
 12  TD_Def            3596 non-null   object        
 13  Sub_Avg           3596 non-null   float64       
 14  _BronzeTimestamp  3596 n

fighter_name           0
Height               263
Weight                74
Reach               1912
Stance               804
DOB                  739
SLpM                   0
Str_Acc                0
SApM                   0
Str_Def                0
TD_Avg                 0
TD_Acc                 0
TD_Def                 0
Sub_Avg                0
_BronzeTimestamp       0
dtype: int64

## Limpieza inicial de `Weight`

- Eliminamos los peleadores que no tienen peso, ya que son muy pocos, y es la información más relevante de un peleador.
- Limpiamos el formato textual (`"185 lbs."` → `185`) y lo convertimos a **entero**.

In [203]:
# Renombramos el DataFrame principal para proteger el dataset original de cambios accidentales mientras trabajas con él.
bronze_clean_df = bronze_df.copy()

# Eliminamos filas sin valor en Weight
bronze_clean_df = bronze_clean_df.dropna(subset=["Weight"])

# Limpiamos el texto dentro de la columna
# Convertimos todos los valores a string, quitamos 'lbs.' (o 'lbs') y espacios en blanco.
# Así pasamos de formatos como '185 lbs.' → '185'
bronze_clean_df["Weight"] = (
    bronze_clean_df["Weight"]
    .astype(str)
    .str.replace("lbs.", "", regex=False)
    .str.strip()
)
# Convertimos el resultado a número entero
# pd.to_numeric convierte los valores a número (si no puede, los pone como NaN).
# astype(int) transforma finalmente a tipo entero.
bronze_clean_df["Weight"] = pd.to_numeric(bronze_clean_df["Weight"], errors="coerce").astype(int)

# Verificamos el resultado
bronze_clean_df.isna().sum()



fighter_name           0
Height               195
Weight                 0
Reach               1838
Stance               739
DOB                  667
SLpM                   0
Str_Acc                0
SApM                   0
Str_Def                0
TD_Avg                 0
TD_Acc                 0
TD_Def                 0
Sub_Avg                0
_BronzeTimestamp       0
dtype: int64

## Transformación del formato de `Str_Acc`, `Str_Def`, `TD_Acc`, `TD_Def`

Necesitamos cambiar el formato de estas 4 columnas de **object** a **float64** de cara a poder eliminar todas las estadísticas vacías
y tabajar en un futuro con ellas.
- Se eliminan símbolos de `%` si existieran.
- Los valores no válidos pasan a `NaN` (coerce).

In [204]:
bronze_clean_df = bronze_clean_df.copy()

# Definimos las columnas que queremos convertir a float
cols_to_convert = ["Str_Acc", "Str_Def", "TD_Acc", "TD_Def"]

# Recorremos cada columna para limpiarla y convertirla
for col in cols_to_convert:
    if col in bronze_clean_df.columns:
         # --- Limpieza del texto ---
         # Convertimos a string, eliminamos el símbolo '%' si existe y espacios sobrantes.
        bronze_clean_df[col] = (
            bronze_clean_df[col]
            .astype(str)
            .str.replace("%", "", regex=False)  # elimina el %
            .str.strip()                        # elimina espacios
        )

        # Convertimos los valores limpios a tipo float64.
        # Si algún valor no se puede convertir (ej. texto no numérico), se reemplaza por NaN.
        bronze_clean_df[col] = pd.to_numeric(bronze_clean_df[col], errors="coerce").astype("float64")

# Verificamos que las columnas se hayan convertido correctamente a float64
print(bronze_clean_df[cols_to_convert].dtypes)

# Mostramos las primeras filas para comprobar los resultados
bronze_clean_df[cols_to_convert].head()

Str_Acc    float64
Str_Def    float64
TD_Acc     float64
TD_Def     float64
dtype: object


,Str_Acc,Str_Def,TD_Acc,TD_Def
0,0.0,0.0,0.0,0.0
1,55.0,48.0,57.0,50.0
2,44.0,58.0,24.0,47.0
3,38.0,57.0,0.0,77.0
4,36.0,63.0,0.0,33.0


## Eliminación de filas sin estadísticas

Se eliminan registros donde **todas** estas métricas valen `0`:
`SLpM`, `Str_Acc`, `SApM`, `Str_Def`, `TD_Avg`, `TD_Acc`, `TD_Def`, `Sub_Avg`.

No aportan información analítica.

In [205]:
bronze_clean_df = bronze_clean_df.copy()

# Definimos las columnas de métricas de rendimiento
# Estas columnas recogen datos de precisión y defensa (golpes, derribos, sumisiones…)
metric_cols = ["SLpM", "Str_Acc", "SApM", "Str_Def", "TD_Avg", "TD_Acc", "TD_Def", "Sub_Avg"]

# Contamos cuántas filas tienen todas estas métricas iguales a 0
# Usamos .apply por filas (axis=1) para comprobar si todas las columnas listadas valen 0 en esa fila.
print("Filas con todas las métricas = 0:", len(bronze_clean_df[bronze_clean_df[metric_cols].apply(lambda fila: all(fila == 0), axis=1)]))

# Eliminamos directamente las filas donde todas las métricas son 0
bronze_clean_df = bronze_clean_df.drop(bronze_clean_df[bronze_clean_df[metric_cols].apply(lambda fila: all(fila == 0), axis=1)].index)

# Mostramos una vista previa del DataFrame resultante
bronze_clean_df.head()

# Mostramos información general del DataFrame
# .info() muestra tipos de datos y número de valores no nulos por columna.
bronze_clean_df.info()

# Revisamos cuántos valores nulos quedan en cada columna
bronze_clean_df.isna().sum()

Filas con todas las métricas = 0: 577
<class 'pandas.core.frame.DataFrame'>
Index: 2945 entries, 1 to 3595
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fighter_name      2945 non-null   object        
 1   Height            2884 non-null   object        
 2   Weight            2945 non-null   int64         
 3   Reach             1674 non-null   object        
 4   Stance            2600 non-null   object        
 5   DOB               2609 non-null   object        
 6   SLpM              2945 non-null   float64       
 7   Str_Acc           2945 non-null   float64       
 8   SApM              2945 non-null   float64       
 9   Str_Def           2945 non-null   float64       
 10  TD_Avg            2945 non-null   float64       
 11  TD_Acc            2945 non-null   float64       
 12  TD_Def            2945 non-null   float64       
 13  Sub_Avg           2945 non-null   float64    

fighter_name           0
Height                61
Weight                 0
Reach               1271
Stance               345
DOB                  336
SLpM                   0
Str_Acc                0
SApM                   0
Str_Def                0
TD_Avg                 0
TD_Acc                 0
TD_Def                 0
Sub_Avg                0
_BronzeTimestamp       0
dtype: int64

## Renombrar `Reach` a `Weight_Class` y clasificar

- Renombramos la columna `Reach` a `Weight_Class` **sin mover su posición**.
- Rellenamos `Weight_Class` según `Weight` (libras), con bins:
  - `(-inf,115]` Strawweight
  - `(115,125]` Flyweight
  - `(125,135]` Bantamweight
  - `(135,145]` Featherweight
  - `(145,155]` Lightweight
  - `(155,170]` Welterweight
  - `(170,185]` Middleweight
  - `(185,205]` Light Heavyweight
  - `(205,inf)` Heavyweight

Como se podía ver la variable `Reach` tenía muchos valores nulos por lo que cuando ocurre esto con una columna lo mejor es eliminarla. Aprovechando esto creamos la columna `Weight_Class` que nos será muy útil de cara a un futuro análisis.

In [206]:
bronze_clean_df = bronze_clean_df.copy()

# Esto aprovecha la posición original de 'Reach' en la tabla para colocar ahí la nueva columna.
bronze_clean_df.rename(columns={"Reach": "Weight_Class"}, inplace=True)

# Definimos los rangos (bins) de peso y las etiquetas (labels) correspondientes
# Cada intervalo representa una categoría oficial de peso en la UFC.
bins = [-float("inf"), 115, 125, 135, 145, 155, 170, 185, 205, float("inf")]
labels = [
    "Strawweight",
    "Flyweight",
    "Bantamweight",
    "Featherweight",
    "Lightweight",
    "Welterweight",
    "Middleweight",
    "Light Heavyweight",
    "Heavyweight",
]

# Asignamos a cada luchador su clase de peso según su valor en 'Weight'
# pd.cut clasifica los valores numéricos de 'Weight' en los intervalos definidos en 'bins'
bronze_clean_df["Weight_Class"] = pd.cut(bronze_clean_df["Weight"], bins=bins, labels=labels, right=True)

# Convertimos la columna a tipo 'category'
# Esto optimiza el uso de memoria y facilita análisis por categorías.
bronze_clean_df["Weight_Class"] = bronze_clean_df["Weight_Class"].astype("category")

# Comprobación rápida
print(bronze_clean_df[["Weight", "Weight_Class"]].head())
bronze_clean_df.info()
bronze_clean_df.isna().sum()


   Weight   Weight_Class
1     185   Middleweight
2     235    Heavyweight
3     155    Lightweight
4     145  Featherweight
5     185   Middleweight
<class 'pandas.core.frame.DataFrame'>
Index: 2945 entries, 1 to 3595
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fighter_name      2945 non-null   object        
 1   Height            2884 non-null   object        
 2   Weight            2945 non-null   int64         
 3   Weight_Class      2945 non-null   category      
 4   Stance            2600 non-null   object        
 5   DOB               2609 non-null   object        
 6   SLpM              2945 non-null   float64       
 7   Str_Acc           2945 non-null   float64       
 8   SApM              2945 non-null   float64       
 9   Str_Def           2945 non-null   float64       
 10  TD_Avg            2945 non-null   float64       
 11  TD_Acc            2945 non-null   float64

fighter_name          0
Height               61
Weight                0
Weight_Class          0
Stance              345
DOB                 336
SLpM                  0
Str_Acc               0
SApM                  0
Str_Def               0
TD_Avg                0
TD_Acc                0
TD_Def                0
Sub_Avg               0
_BronzeTimestamp      0
dtype: int64

## Eliminación de filas sin `Height`

Al igual que ocurre con el peso, la altura es una variable clave. Eliminamos filas con `Height = NaN`.


In [207]:
bronze_clean_df = bronze_clean_df.copy()

# Eliminamos las columnas en las que Height sea vacío
bronze_clean_df = bronze_clean_df.dropna(subset=["Height"])

# Comprobación rápida
bronze_clean_df.info()
bronze_clean_df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 2884 entries, 1 to 3595
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fighter_name      2884 non-null   object        
 1   Height            2884 non-null   object        
 2   Weight            2884 non-null   int64         
 3   Weight_Class      2884 non-null   category      
 4   Stance            2584 non-null   object        
 5   DOB               2598 non-null   object        
 6   SLpM              2884 non-null   float64       
 7   Str_Acc           2884 non-null   float64       
 8   SApM              2884 non-null   float64       
 9   Str_Def           2884 non-null   float64       
 10  TD_Avg            2884 non-null   float64       
 11  TD_Acc            2884 non-null   float64       
 12  TD_Def            2884 non-null   float64       
 13  Sub_Avg           2884 non-null   float64       
 14  _BronzeTimestamp  2884 non-nu

fighter_name          0
Height                0
Weight                0
Weight_Class          0
Stance              300
DOB                 286
SLpM                  0
Str_Acc               0
SApM                  0
Str_Def               0
TD_Avg                0
TD_Acc                0
TD_Def                0
Sub_Avg               0
_BronzeTimestamp      0
dtype: int64

## Eliminación de filas sin `DOB`

Esto será clave de cara a una futura transformación que haremos para conocer la **Edad**.

In [208]:
bronze_clean_df = bronze_clean_df.copy()

# Eliminamos las columnas en las que DOB sea vacío
bronze_clean_df = bronze_clean_df.dropna(subset=["DOB"])

#Comprobación rápida
bronze_clean_df.info()
bronze_clean_df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 2598 entries, 1 to 3595
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fighter_name      2598 non-null   object        
 1   Height            2598 non-null   object        
 2   Weight            2598 non-null   int64         
 3   Weight_Class      2598 non-null   category      
 4   Stance            2404 non-null   object        
 5   DOB               2598 non-null   object        
 6   SLpM              2598 non-null   float64       
 7   Str_Acc           2598 non-null   float64       
 8   SApM              2598 non-null   float64       
 9   Str_Def           2598 non-null   float64       
 10  TD_Avg            2598 non-null   float64       
 11  TD_Acc            2598 non-null   float64       
 12  TD_Def            2598 non-null   float64       
 13  Sub_Avg           2598 non-null   float64       
 14  _BronzeTimestamp  2598 non-nu

fighter_name          0
Height                0
Weight                0
Weight_Class          0
Stance              194
DOB                   0
SLpM                  0
Str_Acc               0
SApM                  0
Str_Def               0
TD_Avg                0
TD_Acc                0
TD_Def                0
Sub_Avg               0
_BronzeTimestamp      0
dtype: int64

## Eliminación de filas sin `Stance`

Tras un rápido muestreo, vemos que la mayoría de peleadores cuyo `Stance` es vacío ni si quiera han 
peleado en **UFC** por lo que los eliminamos.

In [209]:
bronze_clean_df = bronze_clean_df.copy()

# Eliminamos las columnas en las que DOB sea vacío
bronze_clean_df = bronze_clean_df.dropna(subset=["Stance"])

#Comprobación rápida
bronze_clean_df.info()
bronze_clean_df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 2404 entries, 1 to 3594
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fighter_name      2404 non-null   object        
 1   Height            2404 non-null   object        
 2   Weight            2404 non-null   int64         
 3   Weight_Class      2404 non-null   category      
 4   Stance            2404 non-null   object        
 5   DOB               2404 non-null   object        
 6   SLpM              2404 non-null   float64       
 7   Str_Acc           2404 non-null   float64       
 8   SApM              2404 non-null   float64       
 9   Str_Def           2404 non-null   float64       
 10  TD_Avg            2404 non-null   float64       
 11  TD_Acc            2404 non-null   float64       
 12  TD_Def            2404 non-null   float64       
 13  Sub_Avg           2404 non-null   float64       
 14  _BronzeTimestamp  2404 non-nu

fighter_name        0
Height              0
Weight              0
Weight_Class        0
Stance              0
DOB                 0
SLpM                0
Str_Acc             0
SApM                0
Str_Def             0
TD_Avg              0
TD_Acc              0
TD_Def              0
Sub_Avg             0
_BronzeTimestamp    0
dtype: int64

## Transformación de `DOB` a `Age` (edad al 21/03/2021)

- Renombramos `DOB` → `Age`.
- Convertimos a fecha y calculamos la edad exacta en años al **21/03/2021**.
- Guardamos como entero 


In [210]:
bronze_clean_df = bronze_clean_df.copy()
# Renombramos la columna
bronze_clean_df.rename(columns={"DOB": "Age"}, inplace=True)

# Convertimos DOB a tipo datetime
bronze_clean_df["Age"] = pd.to_datetime(bronze_clean_df["Age"], format="%b %d, %Y", errors="coerce")

# Eliminamos filas donde la fecha de nacimiento no sea válida
bronze_clean_df = bronze_clean_df.dropna(subset=["Age"])

# Calculamos la edad al momento de la toma de datos (21 de marzo de 2021)
fecha_corte = datetime(2021, 3, 21)
bronze_clean_df["Age"] = bronze_clean_df["Age"].apply(lambda fecha: (fecha_corte - fecha).days // 365)

# Convertimos la edad a tipo entero normal (sin NA)
bronze_clean_df["Age"] = bronze_clean_df["Age"].astype(int)



In [211]:
# Comprobamos el resultado
print(bronze_clean_df)

             fighter_name  Height  Weight   Weight_Class    Stance  Age  SLpM  \
1              Papy Abedi  5' 11"     185   Middleweight  Southpaw   42  2.80   
2     Shamil Abdurakhimov   6' 3"     235    Heavyweight  Orthodox   39  2.45   
3            Danny Abbadi  5' 11"     155    Lightweight  Orthodox   37  3.29   
5           Ricardo Abreu  5' 11"     185   Middleweight  Orthodox   36  3.79   
6              Daichi Abe  5' 11"     170   Welterweight  Orthodox   29  3.80   
...                   ...     ...     ...            ...       ...  ...   ...   
3585      Errol Zimmerman   6' 3"     185   Middleweight  Orthodox   34  2.95   
3586          Cat Zingano   5' 6"     145  Featherweight  Southpaw   38  2.57   
3587        Joao Zeferino  5' 11"     170   Welterweight  Orthodox   35  0.83   
3591        Zhang Tiequan   5' 8"     155    Lightweight  Orthodox   42  1.23   
3594         Allan Zuniga   5' 7"     155    Lightweight  Orthodox   28  3.93   

      Str_Acc  SApM  Str_De

## Convertir `Height` de pulgadas a centímetros

- Parseamos pies y pulgadas.
- Convertimos a centímetros (`1 in = 2.54 cm`).
- Guardamos como `float`.

In [212]:
bronze_clean_df = bronze_clean_df.copy()

# Definimos una función que transforma alturas tipo pies → centímetros
def height_to_cm(height_str):
    # Convertimos a string y limpiamos espacios
    height_str = str(height_str).strip()

    # --- Búsqueda de pies y pulgadas con expresión regular ---
    # El patrón ^(\d+)'\s*(\d+)? busca:
    #   - Un número entero (pies) seguido de '
    #   - Un número opcional (pulgadas) después del espacio
    match = re.match(r"^(\d+)'\s*(\d+)?", height_str)

    # Si no hay coincidencia (formato distinto o nulo), devolvemos NaN
    if not match:
        return np.nan

    # --- Extracción de pies y pulgadas ---
    feet = int(match.group(1))  # primera parte: pies
    inches = int(match.group(2)) if match.group(2) else 0 # segunda parte: pulgadas

    # --- Conversión total a centímetros ---
    # 1 pie = 12 pulgadas → multiplicamos y luego pasamos a cm (1 in = 2.54 cm)
    total_inches = feet * 12 + inches
    cm = round(total_inches * 2.54)
    return cm

# Aplicamos la función
bronze_clean_df["Height"] = bronze_clean_df["Height"].apply(height_to_cm)

# Convertimos a tipo numérico float
bronze_clean_df["Height"] = bronze_clean_df["Height"].astype(int)


In [213]:
#Comprobación rápida
bronze_clean_df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 2404 entries, 1 to 3594
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fighter_name      2404 non-null   object        
 1   Height            2404 non-null   int64         
 2   Weight            2404 non-null   int64         
 3   Weight_Class      2404 non-null   category      
 4   Stance            2404 non-null   object        
 5   Age               2404 non-null   int64         
 6   SLpM              2404 non-null   float64       
 7   Str_Acc           2404 non-null   float64       
 8   SApM              2404 non-null   float64       
 9   Str_Def           2404 non-null   float64       
 10  TD_Avg            2404 non-null   float64       
 11  TD_Acc            2404 non-null   float64       
 12  TD_Def            2404 non-null   float64       
 13  Sub_Avg           2404 non-null   float64       
 14  _BronzeTimestamp  2404 non-nu

## Comprobación de duplicados en `fighter_name`

In [214]:
# Contamos las apariciones de cada fighter_name
duplicados = bronze_clean_df["fighter_name"].value_counts()
print(duplicados)

fighter_name
Papy Abedi             1
Ode Osbourne           1
Nick Osipczak          1
Dustin Ortiz           1
Nissen Osterneck       1
                      ..
Chris Gruetzemacher    1
Vik Grujic             1
Mike Grundy            1
Leonardo Guimaraes     1
Allan Zuniga           1
Name: count, Length: 2404, dtype: int64


## Eliminación de outliers: `Weight > 265 lbs`

Cualquier registro por encima de 265 lbs se considera fuera de rango reglamentario del
peso pesado → se elimina.

In [215]:
# Contamos los peleadores que hay por cada peso
bronze_clean_df['Weight'].value_counts(dropna=False)

Weight
170    390
155    376
185    305
145    272
135    266
      ... 
295      1
219      1
266      1
244      1
165      1
Name: count, Length: 72, dtype: int64

In [216]:
# Filtramos los registros donde el peso (Weight) supera las 265 libras
outliers_peso = bronze_clean_df[bronze_clean_df["Weight"] > 265]

# Mostramos los posibles outliers encontrados
display(outliers_peso[["fighter_name", "Weight", "Weight_Class"]])

,fighter_name,Weight,Weight_Class
90,Jimmy Ambriz,315,Heavyweight
298,Dan Bobish,345,Heavyweight
543,Hong Man Choi,330,Heavyweight
901,Eric Esch,350,Heavyweight
1633,Mark Kerr,266,Heavyweight
1777,Tae Hyun Lee,295,Heavyweight
1876,Alexandru Lungu,385,Heavyweight
1986,Wagner da Conceicao Martins,390,Heavyweight
2115,Henry Miller,270,Heavyweight
2178,Ricardo Morais,270,Heavyweight


In [217]:
bronze_clean_df = bronze_clean_df.copy()

# Quitamos los peleadores por encima de 265 lbs
bronze_clean_df = bronze_clean_df[bronze_clean_df["Weight"] <= 265]

In [218]:
#Comprobación rápida
print(bronze_clean_df)
bronze_clean_df.describe()

             fighter_name  Height  Weight   Weight_Class    Stance  Age  SLpM  \
1              Papy Abedi     180     185   Middleweight  Southpaw   42  2.80   
2     Shamil Abdurakhimov     190     235    Heavyweight  Orthodox   39  2.45   
3            Danny Abbadi     180     155    Lightweight  Orthodox   37  3.29   
5           Ricardo Abreu     180     185   Middleweight  Orthodox   36  3.79   
6              Daichi Abe     180     170   Welterweight  Orthodox   29  3.80   
...                   ...     ...     ...            ...       ...  ...   ...   
3585      Errol Zimmerman     190     185   Middleweight  Orthodox   34  2.95   
3586          Cat Zingano     168     145  Featherweight  Southpaw   38  2.57   
3587        Joao Zeferino     180     170   Welterweight  Orthodox   35  0.83   
3591        Zhang Tiequan     173     155    Lightweight  Orthodox   42  1.23   
3594         Allan Zuniga     170     155    Lightweight  Orthodox   28  3.93   

      Str_Acc  SApM  Str_De

,Height,Weight,Age,SLpM,Str_Acc,SApM,Str_Def,TD_Avg,TD_Acc,TD_Def,Sub_Avg,_BronzeTimestamp
count,2385.000000,2385.000000,2385.000000,2385.000000,2385.000000,2385.000000,2385.000000,2385.000000,2385.000000,2385.000000,2385.000000,2385
mean,178.333333,168.973585,36.096436,2.925044,42.726625,3.631338,52.124948,1.444327,33.204193,48.863312,0.710063,2025-11-10 13:49:51.311624
min,152.000000,115.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2025-11-10 13:49:51.311624
25%,173.000000,145.000000,31.000000,1.780000,36.000000,2.310000,46.000000,0.000000,0.000000,27.000000,0.000000,2025-11-10 13:49:51.311624
50%,178.000000,168.000000,36.000000,2.760000,43.000000,3.160000,54.000000,1.010000,33.000000,54.000000,0.300000,2025-11-10 13:49:51.311624
75%,185.000000,185.000000,41.000000,3.770000,50.000000,4.280000,60.000000,2.140000,50.000000,71.000000,0.900000,2025-11-10 13:49:51.311624
max,211.000000,265.000000,64.000000,19.910000,100.000000,52.500000,100.000000,24.110000,100.000000,100.000000,21.900000,2025-11-10 13:49:51.311624
std,8.641732,35.297319,6.811577,1.681676,12.424499,2.630241,11.693850,1.684660,27.652851,30.944469,1.401760,NaN


## Renombramos columnas: `Height`, `Weight`, `Str_Acc`, `Str_Def`, `TD_Acc`, `TD_Def`

Esto de cara a que cualquiera pueda entender las unidades en que se expresan los datos de la tabla.

In [219]:
# Reenombramos columnas
bronze_clean_df.rename(columns={"Height": "Height(cm)"}, inplace=True)
bronze_clean_df.rename(columns={"Weight": "Weight(lbs)"}, inplace=True)
bronze_clean_df.rename(columns={"Str_Acc": "Str_Acc(%)"}, inplace=True)
bronze_clean_df.rename(columns={"Str_Def": "Str_Def(%)"}, inplace=True)
bronze_clean_df.rename(columns={"TD_Acc": "TD_Acc(%)"}, inplace=True)
bronze_clean_df.rename(columns={"TD_Def": "TD_Def(%)"}, inplace=True)

## Trazabilidad: `_SilverTimestamp`

Añadimos una marca temporal con la fecha/hora de generación de esta versión **Silver**.

In [220]:
# Añadimos el Timestampo de la capa Silver
bronze_clean_df["_SilverTimestamp"] = datetime.now()

In [221]:
# Comprobación rápida
bronze_clean_df.head()

,fighter_name,Height(cm),Weight(lbs),Weight_Class,Stance,Age,SLpM,Str_Acc(%),SApM,Str_Def(%),TD_Avg,TD_Acc(%),TD_Def(%),Sub_Avg,_BronzeTimestamp,_SilverTimestamp
1,Papy Abedi,180,185,Middleweight,Southpaw,42,2.80,55.0,3.15,48.0,3.47,57.0,50.0,1.3,2025-11-10 13:49:51.311624,2025-11-10 20:39:06.925614
2,Shamil Abdurakhimov,190,235,Heavyweight,Orthodox,39,2.45,44.0,2.45,58.0,1.23,24.0,47.0,0.2,2025-11-10 13:49:51.311624,2025-11-10 20:39:06.925614
3,Danny Abbadi,180,155,Lightweight,Orthodox,37,3.29,38.0,4.41,57.0,0.00,0.0,77.0,0.0,2025-11-10 13:49:51.311624,2025-11-10 20:39:06.925614
5,Ricardo Abreu,180,185,Middleweight,Orthodox,36,3.79,31.0,3.98,68.0,2.13,42.0,100.0,0.7,2025-11-10 13:49:51.311624,2025-11-10 20:39:06.925614
6,Daichi Abe,180,170,Welterweight,Orthodox,29,3.80,33.0,4.49,56.0,0.33,50.0,0.0,0.0,2025-11-10 13:49:51.311624,2025-11-10 20:39:06.925614


## Guardado del dataset Silver

Guardamos el parquet `fighter_details_silver.parquet`.


In [222]:
# Guardamos el DataFrame en formato Parquet en la ruta de BRONZE definida
bronze_clean_df.to_parquet(silver_file_path, index=False)